In [1]:
import os
import time

import isx  #Inscopix API
import numpy as np

### User Configurables

In [2]:
projectName = 'VGAT4-Session1'
data_dir = 'D:\VGAT4-Session1'
output_dir = 'D:\VGAT4-Session1\DataAnalysis'
video_base = '2022-10-17-13-45-14_video'

trials = 1
efocus = [250]  #If these values do not match the efocus used in the exp, code will crash

#Downsample Settings
spatial_downsample_factor = 4 # Factor of 4 is recommended for CNMFE

#Spatial Bandpass Settings
SBP_low_cutoff = 0.005
SBP_high_cutoff = 0.500

#Motion Correction Settings
max_translation = 20
MC_low_bandpass_cutoff = None
MC_high_bandpass_cutoff = None

#CNMFE Settings
cell_diameter = 7
merge_threshold = 0.7 #default is 0.7, use default when you have a 10-min video for baseline
guassian_kernel_size = 3
output_unit_type = 0
min_corr = 0.9
min_pnr = 15

### Some processing goes on here

In [3]:
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

proc_move_files = []
proc_cs_files = []

vidfiles = []

# Automatically obtain the file names for the videos
if trials > 1:
    for i in range(trials-1):
        filename = f'{video_base}{str(i)}.isxd'
        videopath = os.path.join(output_dir, filename)
        vidfiles.append(videopath)
elif trials == 1:
    filename = f'{video_base}.isxd'
    videopath = os.path.join(data_dir, filename)
    vidfiles.append(videopath)


if len(efocus) > 1:
    deinterlace_files_matrix = np.array((len(efocus), trials))
    for i in range(len(efocus)):
        deinterlace_files_matrix[i,:] = isx.make_output_file_paths(vidfiles, output_dir, str(efocus(i)))
    deinterlace_files = np.reshape(deinterlace_files_matrix, (1, len(efocus)*trials))
    
    isx.deinterleave(vidfiles, deinterlace_files,efocus)
    
    series_rec_names = {}
    for i in range(len(efocus)):
        keyName = f'focal{efocus[i]}'
        series_rec_names[keyName] = deinterlace_files[i,:]
    series_rec_names_keys = list(series_rec_names.keys())
else:
    series_rec_names = {}
    keyName = f'focal{efocus[0]}'
    series_rec_names[keyName] = vidfiles
    series_rec_names_keys = list(series_rec_names.keys())
    

In [4]:
def check_files(filelist):
    for files in filelist:
        if not os.path.exists(files) or not os.path.getsize(files) > 2048:
            return False

    return True   

In [5]:
for key in range(len(series_rec_names_keys)):
    series_name = series_rec_names_keys[key]
    rec_files = series_rec_names[series_name]
    
    pp_files = isx.make_output_file_paths(rec_files, output_dir, 'PP')
    if not check_files(pp_files):
        print('Starting Preprocessing...')
        isx.preprocess(rec_files, pp_files, spatial_downsample_factor=spatial_downsample_factor)
    
    bp_files = isx.make_output_file_paths(pp_files, output_dir, 'BP')
    if not check_files(bp_files):
        print('Starting Bandpass Filtering...')
        isx.spatial_filter(pp_files, bp_files, low_cutoff=SBP_low_cutoff, high_cutoff=SBP_high_cutoff)
    
    mean_proj_file = os.path.join(output_dir, f'{series_name}-mean_image.isxd')
    if not os.path.exists(mean_proj_file):
        print('Creating Mean Image...')
        isx.project_movie(bp_files, mean_proj_file, stat_type='mean')
    
    mc_files = isx.make_output_file_paths(bp_files, output_dir, 'MC')
    translation_files = isx.make_output_file_paths(mc_files, output_dir, 'translations', 'csv')
    crop_rect_file = os.path.join(output_dir, f'{series_name}-crop_rect.csv')
    
    if not check_files(mc_files):
        print('Starting Motion Correction...')
        isx.motion_correct(bp_files, mc_files, max_translation=max_translation, reference_file_name=mean_proj_file,
                           low_bandpass_cutoff=MC_low_bandpass_cutoff, high_bandpass_cutoff=MC_high_bandpass_cutoff,
                           output_translation_files=translation_files, output_crop_rect_file=crop_rect_file)
    
    dff_files = isx.make_output_file_paths(mc_files, output_dir, 'DFF')

    if not check_files(dff_files):
        print('Calculating DF/F...')
        isx.dff(mc_files, dff_files, f0_type='mean')

    cnmfe_files = isx.make_output_file_paths(mc_files, output_dir, 'CNMFE')


    if not check_files(mc_files) or not check_files(cnmfe_files):
        print('Running CNMFe (This Will Take a Long Time)...')
        startTime = time.time()
        isx.run_cnmfe(mc_files, cnmfe_files, output_dir, cell_diameter=cell_diameter, merge_threshold=merge_threshold, gaussian_kernel_size=guassian_kernel_size,
                       output_unit_type='df', min_corr=min_corr, min_pnr=min_pnr, processing_mode='parallel_patches', num_threads=8)
        endTime = time.time()
        print(f'Built-in CNMFE Total Elapsed Time: {(endTime-startTime)/60} (min)')

    movie_filename = f'MC_ForCNMFE{efocus[key]}.tiff'
    tiff_movie_file = os.path.join('C:\\Users\\dewanlab.PSY-PDB-C474-B\Desktop', movie_filename)
   # cnmfe_dir = os.path.join('C:\\Users\\dewanlab.PSY-PDB-C474-B\Desktop', 'CNMFE')
   #  if not check_files(tiff_movie_file):
   #      print('Running CNMFe Module (This Will Take a Long Time)...')
   #      startTime = time.time()
   #      #isx_cnmfe.run_cnmfe(tiff_movie_file, cnmfe_dir, average_cell_diameter=cell_diameter, merge_threshold=merge_threshold, gaussian_kernel_size=guassian_kernel_size,
   #      #              output_units=0, min_pixel_correlation=min_corr, min_peak_to_noise_ratio=min_pnr, processing_mode=2, num_threads=8, verbose=1)
   #      endTime = time.time()
   #      print(f'CNMFE Module Total Elapsed Time: {(endTime-startTime)/60} (min)')

    contour_files = isx.make_output_file_paths(dff_files, output_dir, 'AC')

    if not check_files(dff_files) or not check_files(cnmfe_files) or not check_files(contour_files):
        print('Applying Cell Contours...')
        isx.apply_cell_set(dff_files, cnmfe_files[0], contour_files, 0)

    tiffFileName = f'MC-DFF-AC-TIFF-{series_name}.tiff'
    csvFileName = f'MC-DFF-AC-{series_name}.csv'
    jsonFileName = f'CONTOUR-{series_name}.json'
    csvPropsFileName = f'MC-DFF-AC-{series_name}-Prop.csv'
    gpioInputFileName = f'{video_base}.gpio'
    gpioOutputFileName = f'GPIO-{series_name}.csv'

    tiff_image_file = os.path.join(output_dir, tiffFileName)
    csv_trace_file = os.path.join(output_dir, csvFileName)
    csv_props_file = os.path.join(output_dir, csvPropsFileName)
    print('Exporting Traces and Props...')
    isx.export_cell_set_to_csv_tiff(contour_files, csv_trace_file, tiff_image_file, time_ref= 'unix', output_props_file= csv_props_file)

    json_file = os.path.join(output_dir, jsonFileName)
    print('Exporting Cell Contours...')
    isx.export_cell_contours(cnmfe_files[0], json_file)

    input_GPIO_file = os.path.join(data_dir, gpioInputFileName)
    output_GPIO_file = os.path.join(output_dir, gpioOutputFileName)

    print("Exporting GPIO to csv...")
    isx.export_gpio_set_to_csv(input_GPIO_file, output_GPIO_file, time_ref= 'unix')

    #Graphing stuff here
    movie_filename = f'DFF_{efocus[key]}.mp4'
    movie_file = os.path.join(output_dir, movie_filename)
    print('Exporting Movie to MP4...')
    isx.export_movie_to_mp4(dff_files, movie_file, compression_quality= 0.4, write_invalid_frames= True)
    print('Processing Finished!')


Exporting Traces and Props...
Exporting Cell Contours...
Exporting GPIO to csv...
Exporting Movie to MP4...
Processing Finished!
